IMPORTS

In [31]:
import pandas as pd
import datetime as dt
import numpy as np

BASES

In [32]:
# LIEN ACTEURS

# df_name_basics = pd.read_csv('BD/name.basics.tsv.gz', compression = 'gzip', sep = '\t')
# df_title_principals = pd.read_csv('BD/title.principals.tsv.gz', compression = 'gzip', sep = '\t')
# df_title_crew = pd.read_csv('BD/title.crew.tsv.gz', compression = 'gzip', sep = '\t')

MERGES

In [33]:
# CHANGEMENTS DANS BASE TITLE EPISODES

df_title_episodes = pd.read_csv('BD/title.episode.tsv.gz', compression = 'gzip', sep = '\t')

df_title_episodes[['seasonNumber', 'episodeNumber']] = df_title_episodes[['seasonNumber', 'episodeNumber']].replace('\\N', 0)
df_title_episodes[['seasonNumber', 'episodeNumber']] = df_title_episodes[['seasonNumber', 'episodeNumber']].apply(pd.to_numeric)

df_serie = df_title_episodes.groupby('parentTconst').sum()
df_serie = df_serie[['seasonNumber', 'episodeNumber']]
df_serie = df_serie.reset_index()

df_episodes = pd.DataFrame(df_title_episodes['tconst'])

df_serie['titleType'] = 'serie'
df_serie = df_serie.rename(columns = {'parentTconst': 'tconst'})
df_episodes['titleType'] = 'episode'
df_serie = df_serie.rename(columns = {'parentTconst': 'tconst'})
df_title_episodes = pd.concat([df_serie, df_episodes])

df_serie, df_episodes = 0, 0

In [34]:
df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')
df_title_ratings = pd.read_csv('BD/title.ratings.tsv.gz', compression = 'gzip', sep = '\t')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2145/3795572048.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')


In [35]:
df1 = pd.merge(df_title_basics,
               df_title_ratings,
               how = 'outer',
               on = 'tconst')

df_title_ratings, df_title_basics = 0, 0

In [36]:
df2 = pd.merge(df1,
               df_title_episodes,
               how = 'outer',
               on = 'tconst')

df1, df_title_episodes = 0, 0

In [37]:
df_title_akas = pd.read_csv('BD/title.akas.tsv.gz', compression = 'gzip', sep = '\t')

df_3 = pd.merge(df2,
                df_title_akas,
                how = 'outer',
                left_on = 'tconst',
                right_on = 'titleId')

df2, df_title_akas = 0, 0

In [38]:
df_tmbd_full = pd.read_csv('BD/tmdb_full.csv')

df_films = pd.merge(df_tmbd_full,
                df_3,
                how = 'outer',
                left_on = 'imdb_id',
                right_on = 'tconst')

df_tmbd_full, df_3 = 0, 0

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2145/2029903383.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmbd_full = pd.read_csv('BD/tmdb_full.csv')


CLEAN DU MERGE

In [39]:
# DROP CE QUI N'EST PAS FILM OU SERIE

# df_films = df_films[(df_films['titleType_x'] == 'movie') | (df_films['titleType_x'] == 'tvEpisode') | (df_films['titleType_x'] == 'tvSeries') | (df_films['titleType_y'] == 'serie') | (df_films['titleType_y'] == 'episode')]
df_films = df_films[(df_films['titleType_x'] == 'movie')]

In [40]:
# REPLACE \\N

df_films = df_films.replace('\\N', None)

In [41]:
# COLONNES DATES

df_films['release_date'] = pd.to_datetime(df_films['release_date']).dt.year

In [61]:
# COLONNES NUMERIQUES

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].fillna(0, inplace = True)
df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].apply(pd.to_numeric)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_2145/2725286597.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].fillna(0, inplace = True)


In [62]:
# RELEASE YEAR

df_films['year_exact'] = df_films[['release_date', 'startYear']].max(axis = 1)

In [68]:
df_films['Decennie'] = ''
df_films['year_exact'] = df_films['year_exact'].fillna('0')
df_films['year_exact'] = df_films['year_exact'].replace('nan', '0')
df_films['year_exact'] = df_films['year_exact'].astype(str)
df_films[df_films['year_exact'] != '0']['Decennie'] = df_films['year_exact'].str[:3] + "0's"
df_films['year_exact'] = pd.to_numeric(df_films['year_exact'])

In [73]:
# RUNTIME & RATINGS

df_films['difference_runtime'] = abs(df_films['runtime'] - df_films['runtimeMinutes'])
df_films['difference_runtime'] = df_films['difference_runtime'].fillna(0)
df_films['difference_runtime'] = df_films['difference_runtime'].apply(pd.to_numeric)
df_films['runtime_exact'] = ''

df_films['difference_vote'] = abs(df_films['averageRating'] - df_films['vote_average'])
df_films['difference_vote'] = df_films['difference_vote'].fillna(0)
df_films['difference_vote'] = df_films['difference_vote'].apply(pd.to_numeric)
df_films['vote_exact'] = ''


In [74]:
df_films['runtimeMinutes'] = df_films['runtimeMinutes'].apply(pd.to_numeric)
df_films[df_films['difference_runtime'] == 0]['runtime_exact'] = df_films['runtime']
df_films[df_films['difference_runtime'] > 0]['runtime_exact'] = df_films[['runtimeMinutes', 'runtime']].max(axis = 1)  

In [75]:
df_films[df_films['difference_vote'] == 0]['vote_exact'] = df_films['vote_average']
df_films[df_films['difference_vote'] <= 1]['vote_exact'] = df_films[['averageRating', 'vote_average']].mean(axis = 1)
df_films[df_films['difference_vote'] > 1]['vote_exact'] = round(((df_films['vote_count'] * df_films['vote_average']) + (df_films['numVotes'] * df_films['averageRating'])) / (df_films['numVotes'] + df_films['vote_count']), 2)

In [76]:
# GENRES

df_films['total_genres'] = df_films['genres_x'] + df_films['genres_y']

In [77]:
df_films.sample(3).transpose()

,31698113,28744840,45491773
adult,NaN,False,NaN
backdrop_path,NaN,NaN,NaN
budget,NaN,NaN,NaN
genres_x,NaN,['Documentary'],NaN
homepage,NaN,NaN,NaN
id,NaN,217686.0,NaN
imdb_id,NaN,tt2633872,NaN
original_language,NaN,en,NaN
original_title,NaN,Miracle Rising: South Africa,NaN
overview,NaN,MIRACLE RISING: SOUTH AFRICA is the epic legac...,NaN


EXPORT

In [ ]:
# df_films.to_csv('P2_G5_films.csv.gz', index=False, compression='gzip')